# Segmented Ensemble Analysis 

In [ ]:
from os.path import join
import numpy as np
import pandas as pd
from itertools import chain, combinations
from scipy.optimize import curve_fit

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, lasso_path
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
from seaborn import *

plt.rc('axes', grid=False)

## Load Data

In [ ]:
#target file
fnens = join('..', 'data', 'segmented_weathering.csv')
#plots directory
dirplots = join('..', 'plots', 'notebook')
# units of [m]
earth_radius = 6.371e6
# units of [m^2]
earth_surface_area = 4*np.pi*earth_radius**2
#area of indonesia [steradians]
indonesia_area = 1.9e6 * 1e6 / earth_surface_area
#load the data and inspect some rows
ens = pd.read_csv(fnens)
#take the absolute mean latitude, hemisphere doesn't matter
ens['lat'] = ens['meanlat'].apply(np.abs)
ens.drop('meanlat', axis=1, inplace=True)
#rename perimeter and lat
ens.rename(dict(perimeter='P', meanabslat='lat'), axis=1, inplace=True)
#total weathering columns
twcols = ['whak', 'mac']
#average weathering columns
mwcols = ['meanwhak', 'meanmac']
#other columns
vcols = ['lat', 'minlat', 'A', 'P', 'O', 'PAR']
#mean weathering unit conversions
for c in mwcols:
    ens[c] *= 365*24*3600 # mole/m^2/yr
for c in twcols:
    ens[c] *= 365*24*3600/1e12
#ratio of perimeter to root-area, describing the shape
ens['PAR'] = ens.P / ens.A.apply(np.sqrt)
#other unit conversions
ens.A /= earth_surface_area #fraction of Earth surface area
ens.O /= earth_radius #fraction of Earth circumference
ens.P /= earth_radius  #fraction of Earth circumference
ens.q *= 365*24*3600 # m/yr/m^2
#include log10 versions of weathering columns
for c in mwcols + twcols:
    ens['log'+c] = ens[c].apply(np.log)
#also groups for the area and latitude
ens['Area Range'] = pd.cut(ens.A.apply(np.sqrt), 5)
ens['Latitude Range'] = pd.cut(
    ens.lat,
    [0, 15, 30, 45, 65, 90],
    labels=['<15°', '15-30°', '30-45°', '45-65°', '>65°']
)

ens.describe()

## Formatting Dictionaries & Function for Pretty Plots

In [ ]:
#long labels for plotting/reference
labels = dict(
    A='Area [Steradians]',
    O='D [Radians]',
    T='Temperature [K]',
    lat='$\\theta$ [°]',
    minlat='$\\theta_m$ [°]',
    P='Perimeter [radians]',
    q='Runoff [m/yr/m$^2$]',
    PAR='Perimeter / √Area [-]',
    meanwhak='WHAK$_A$ [mole/m$^2$/yr]',
    meanmac='MAC$_A$ [mole/m$^2$/yr]',
    whak='WHAK$_T$ [Tmole/yr]',
    mac='MAC$_T$ [Tmole/yr]'
)

In [ ]:
#standard scatter plot configuration
scatter_kws = dict(
    hue='lat',
    palette='flare',
    size='A',
    legend=False,
    linewidth=0.5,
    edgecolor='k'
)

In [ ]:
box_kws = dict(
    medianprops=dict(
        color='#cecece',
        zorder=101
    ),
    boxprops=dict(
        edgecolor='#808080',
        zorder=100
    ),
    capprops=dict(
        color='#808080'
    ),
    whiskerprops=dict(
        color='#808080'
    ),
    flierprops=dict(
        markerfacecolor='#808080',
        markeredgecolor='#808080'
    )
)

In [ ]:
unitless = lambda label: label[:label.index('[') - 1]

wrapped = lambda label: label.replace(' ','\n')

def ylabel(label: str, ax):
    ax.set_ylabel(label, ha='right', rotation=0)
    ax.yaxis.set_label_coords(0, 1.02)
    return None

lat_transform = lambda theta: np.sin(theta*(np.pi/180))**2

## Size-Latitude Distribution

In [ ]:
jg = jointplot(
    data=ens,
    x='lat',
    y=ens.A,
    color='gray',
    kind='scatter',
    height=6,
    joint_kws=dict(
        edgecolor='k'
    ),
    marginal_kws=dict(
        color='gray',
        edgecolor='k',
        bins=32
    )
)
ax = jg.ax_joint
ax.set_xlabel(labels['lat'])
ax.set_yticks([])
ax.set_ylabel('Area', rotation=0, ha='right')
jg.ax_marg_x.tick_params(axis='x', length=0)
jg.ax_marg_x.spines['bottom'].set_visible(False)
jg.ax_marg_y.set_yticks([])
jg.ax_marg_y.spines['left'].set_visible(False)
jg.figure.savefig(join(dirplots, 'scatter_area'), dpi=500)

## How Much Does Total Weathering Vary With Continental Configuration? 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,5))
df = ens.groupby(['n','p']).sum(numeric_only=True)
for w in twcols:
    df[w] = df[w] - df[w].mean() + 7
df = df.melt(
    value_vars=twcols,
    var_name='formula',
    value_name='weathering'
)
df.formula = df.formula.str.upper()
kdeplot(
    data=df,
    x='weathering',
    hue='formula',
    cut=0,
    common_norm=True,
    bw_adjust=0.75,
    linewidth=2.5,
    ax=ax
)
ax.spines['left'].set_visible(False)
ax.set_ylabel(None)
ax.set_yticks([])
ax.set_xlabel('Total Weathering [Tmole/yr]')
leg = ax.get_legend()
leg.set_frame_on(False)
leg.set_title(None)
fig.tight_layout()
fig.subplots_adjust(wspace=0.2)
fig.savefig(join(dirplots, 'weathering_distributions'), dpi=500)

## How Does Weathering Vary with Geometric Features?

In [ ]:
fig, axs = plt.subplots(2, len(vcols), figsize=(12,5))
for i,w in enumerate(mwcols):
    for j,v in enumerate(vcols):
        scatterplot(
            data=ens,
            x=v,
            y=w,
            ax=axs[i,j],
            **scatter_kws
        )
        axs[i,j].set_yticks([])
        axs[i,j].set_xticks([])
        axs[i,j].set_xlabel(unitless(labels[v]))
        axs[i,j].set_ylabel(None)
    ylabel(wrapped(unitless(labels[w])), axs[i,0])
fig.tight_layout()
fig.savefig(join(dirplots, 'scatters_weathering_mean'), dpi=500)

In [ ]:
fig, axs = plt.subplots(2, len(vcols), figsize=(12,5))
for i,w in enumerate(mwcols):
    for j,v in enumerate(vcols):
        scatterplot(
            data=ens,
            x=v,
            y='log'+w,
            ax=axs[i,j],
            **scatter_kws
        )
        axs[i,j].set_yticks([])
        axs[i,j].set_xticks([])
        axs[i,j].set_xlabel(unitless(labels[v]))
        axs[i,j].set_ylabel(None)
    ylabel(wrapped('log ' + unitless(labels[w])), axs[i,0])
fig.tight_layout()
fig.savefig(join(dirplots, 'scatters_weathering_logmean'), dpi=500)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,5))
for ax,w in zip(axs, mwcols):
    boxplot(
        data=ens,
        x='Area Range',
        y='log'+w,
        hue='Latitude Range',
        palette='flare',
        whis=[0,100],
        ax=ax,
        **box_kws
    )
    move_legend(ax, 'lower right')
    ax.set_xticks([])
    ylabel('log ' + unitless(labels[w]), ax)
    ax.set_xlabel(None)
    ax.grid(False)
    ax.get_legend().set_frame_on(False)
    ax.set_xlabel('Land Area\n⟵ small | large ⟶')
    ax.set_yticks([])
fig.tight_layout()
fig.subplots_adjust(top=0.85)
fig.savefig(join(dirplots, 'boxes_area_avg'), dpi=500)

## Can Average Weathering Be Parameterized by A Simple Function?

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12,8))
X = ens.lat.values.copy()
X /= X.max()
exp_scores = dict()
for i,w in enumerate(mwcols):

    y = ens[w].values
    y = (y - y.min())/(y.max() - y.min())

    f = lambda x, a, b, c: c*np.exp(a*x) + b
    coef, _ = curve_fit(f, X, y, (-10., 0., 1.))
    score = r2_score(y, f(X, *coef))
    exp_scores[w] = score
    print(f'{w} regression score (r^2) = {score}')

    scatterplot(
        data=ens,
        x=X.flatten(),
        y=y,
        ax=axs[i,0],
        **scatter_kws
    )
    x = np.linspace(X.min(), X.max(), 100)
    axs[i,0].plot(x, f(x, *coef), 'w')
    ylabel(unitless(labels[w]), axs[i,0])
    axs[i,0].set_xlabel('$\\theta$')
    axs[i,0].set_xticks([])
    axs[i,0].set_yticks([])

    resid = y - f(X, *coef)
    print(f'{w} MAE =', np.abs(resid).mean())
    print(f'{w} std =', resid.std())
    scatterplot(
        data=ens,
        x=X.flatten(),
        y=resid,
        ax=axs[i,1],
        **scatter_kws
    )
    axs[i,1].plot(x, np.zeros(len(x)), 'w')
    ylabel(unitless(labels[w]) + '\nResidual', axs[i,1])
    axs[i,1].set_xlabel('$\\theta$')
    axs[i,1].set_xticks([])
    axs[i,1].set_yticks([])
fig.tight_layout()
fig.savefig(join(dirplots, 'regression_weathering_exponential'), dpi=500)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12,7), sharey=True, sharex=True)
X = preprocessing.scale(ens[vcols].values)
alphas = 10 ** np.linspace(-4, 0, 101)
for i,log in enumerate(['', 'log']):
    for j,w in enumerate(mwcols):
        ax = axs[i,j]
        y = ens[w].values
        y[y == 0] = y[y != 0].min()
        if log:
            y = preprocessing.scale(np.log(y))
        else:
            y = preprocessing.scale(y)
        alphas, coefs, _ = lasso_path(X, y, alphas=alphas)
        r = ax.pcolormesh(
            alphas,
            range(len(vcols)),
            np.abs(coefs),
            cmap='magma'
        )
        cb = plt.colorbar(r, ax=ax)
        cb.ax.set_ylabel('$|\\beta|$', ha='center', va='bottom', rotation=0)
        cb.ax.yaxis.set_label_coords(0.5, 1.02)
        ax.set_xscale('log')
        ax.set_yticks(range(len(vcols)))
        ax.set_yticklabels([unitless(labels[v]) for v in vcols])
        ax.set_title(log + ' ' + unitless(labels[w]) + ' Lasso Path')
fig.supxlabel('Regularization Coefficient ($\\alpha$)')
fig.tight_layout()
fig.savefig(join(dirplots, "lasso_paths"), dpi=500)

In [ ]:
def normalize_ens(Xcols, ycol, df=ens):
    X = preprocessing.scale(df[Xcols].values)
    y = preprocessing.minmax_scale(df[ycol].values, (-1,1))
    return X, y

def cross_val_MLP(Xcols, ycol, nhidden, nlayer, ntrial, split=0.7, df=ens, **kws):
    X, y = normalize_ens(Xcols, ycol, df)
    scores = cross_val_score(
        MLPRegressor([nhidden]*nlayer, **kws),
        X,
        y,
        cv=ShuffleSplit(
            n_splits=ntrial,
            train_size=split,
            test_size=(1-split)
        ),
        n_jobs=4
    )
    return scores

In [ ]:
mlp_kws = dict(
    learning_rate='adaptive',
    activation='tanh',
    solver='lbfgs',
    max_iter=10_000,
    alpha=0.04,
    tol=1e-8
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,5), sharey=True)
nhiddens = [2, 4, 8, 16]
nlayer = 2
ntrial = 32
regcols = vcols
L = len(nhiddens)*ntrial
scores = pd.DataFrame({
    'nhidden': np.zeros(L, dtype=np.int16),
    'meanwhak': np.zeros(L),
    'meanmac': np.zeros(L)
})
for i,nhidden in enumerate(nhiddens):
    scores.loc[i*ntrial:(i+1)*ntrial-1,'nhidden'] = nhidden
    scores.loc[i*ntrial:(i+1)*ntrial-1,'meanwhak'] = cross_val_MLP(
        regcols, 
        'meanwhak', 
        nhidden, 
        nlayer, 
        ntrial, 
        **mlp_kws
    )
    scores.loc[i*ntrial:(i+1)*ntrial-1,'meanmac'] = cross_val_MLP(
        regcols, 
        'meanmac', 
        nhidden, 
        nlayer, 
        ntrial, 
        **mlp_kws
    )
for w,ax in zip(mwcols, axs):
    boxplot(
        x=scores.nhidden,
        y=scores[w],
        color='gray',
        whis=[0,100],
        ax=ax,
        **box_kws
    )
    ylabel(wrapped(unitless(labels[w]) + ' Regression Score'), ax)
    ax.set_xlabel(None)
    ax.axhline(1, linestyle='--', color='white', linewidth=0.75, alpha=0.75)
fig.supxlabel('Hidden Perceptrons')
fig.savefig(join(dirplots, 'hidden_size_cv'), dpi=500)

In [ ]:
combos = map(lambda r: combinations(vcols[:-2], r), range(1,len(vcols[:-2]) + 1))
combos = sorted(list(chain(*combos)))
yticklabels = [(', '.join(c)).replace('minlat','$\\theta_m$').replace('lat','$\\theta$') for c in combos]
fig, axs = plt.subplots(2, 1, figsize=(6,10), sharey=True)
for w,ax in zip(mwcols, axs):
    scores = {}
    for combo in combos:
        scores[', '.join(combo)] = cross_val_MLP(
            list(combo),
            'meanwhak',
            4, #hidden units
            2, #layers
            24, #trials/splits
            **mlp_kws
        )
    scores = pd.DataFrame(scores)
    scores = scores.melt(
        value_vars=scores.columns,
        var_name='Features',
        value_name='Regression Score'
    )
    barplot(
        scores,
        y='Features',
        x='Regression Score',
        color='gray',
        width=0.7,
        estimator=np.median,
        errorbar='ci',
        errcolor='w',
        ax=ax
    )
    scores = scores.groupby('Features').median()
    maxscore = scores.iloc[scores['Regression Score'].argmax()]
    print(f'{w} best scoring combination (median):\n{maxscore}')
    ax.axvline(exp_scores[w], color='C3', linestyle='--', linewidth=0.75, alpha=0.75, zorder=-1)
    ax.axvline(maxscore.iloc[0], color='w', linestyle='--', linewidth=0.75, alpha=0.5)
    ax.set_xlabel(unitless(labels[w]) + ' Regression Score')
    ax.set_ylabel(None)
    ax.set_xticks([0, maxscore.values[0], 1])
    ax.set_ylabel('Feature Combination')
    ax.set_yticklabels(yticklabels)

fig.tight_layout()
fig.savefig(join(dirplots, 'feature_permutation'), dpi=500)

In [ ]:
def prediction_surface(reg, X, y, ax, cmap='viridis', scatter=True):
    
    x1 = np.linspace(X[:,0].min(), X[:,0].max(), 101)
    x2 = np.linspace(X[:,1].min(), X[:,1].max(), 101)
    x1g, x2g = np.meshgrid(x1, x2)
    preds = reg.predict(np.stack([x1g.flatten(), x2g.flatten()]).T).reshape(101,101)
    r = ax.pcolormesh(
        x1,
        x2,
        preds,
        shading='gouraud',
        cmap=cmap,
        vmin=y.min(),
        vmax=y.max(),
        zorder=1
    )
    cb = plt.colorbar(r, ax=ax)
    cb.set_label(unitless(labels[w]), rotation=270, va='bottom')
    cb.set_ticks([])
    ax.contour(
        x1,
        x2,
        preds,
        colors='k',
        linewidths=0.75,
        linestyles='-',
        zorder=2
    )
    if scatter:
        ax.scatter(
            X[:,0],
            X[:,1],
            c=y,
            s=10,
            edgecolors='none',
            cmap=cmap,
            vmin=y.min(),
            vmax=y.max(),
            zorder=3
        )
    ax.set_xlabel(unitless(labels[regcols[0]]))
    ax.set_xticks([])
    ylabel(unitless(labels[regcols[1]]), ax)
    ax.set_yticks([])
    
    return None

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12,6))
cmap = 'viridis'
regcols = ['lat', 'A']
nlayer = 2
nhidden = 4
models = dict()
for i,w in enumerate(mwcols):

    X, y = normalize_ens(regcols, w, ens)
    reg = MLPRegressor([nhidden]*nlayer, **mlp_kws).fit(X, y)
    models[w] = reg

    prediction_surface(reg, X, y, axs[i,0], cmap)

    ax = axs[i,1]
    scatterplot(
        data=ens,
        x=y,
        y=reg.predict(X),
        ax=ax,
        **scatter_kws
    )
    ax.set_xlabel('Actual ' + unitless(labels[w]))
    ylabel('Predicted\n' + unitless(labels[w]), ax)
    ax.plot(
        [y.min(), y.max()],
        [y.min(), y.max()],
        linestyle=':',
        color='w',
        linewidth=1,
        zorder=10
    )
    ax.set_yticks([])
    ax.set_xticks([])
    ax.axis('equal')

    ax = axs[i,2]
    resid = y - reg.predict(X)
    print(w, 'score =', reg.score(X, y))
    mae = np.abs(resid).mean()
    print(w, f'MAE = {mae} ({100*mae/(y.max() - y.min())} % of range)')
    print(w, 'std =', resid.std())
    kdeplot(
        x=resid,
        color='w',
        ax=ax
    )
    ax.axvline(0, linestyle=':', color='w', linewidth=1, zorder=10)
    ax.set_ylabel(None)
    ax.set_xlabel(unitless(labels[w]) + ' Residual')
    ax.set_yticks([])
    ax.set_xticks([])

axs[1,-1].set_xlim(axs[0,-1].get_xlim())
fig.tight_layout()
fig.savefig(join(dirplots, 'regression_MLP'), dpi=500)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,5))
for w,ax in zip(mwcols, axs):
    X, y = normalize_ens(regcols, w, ens)
    idx = ens.lat.values > 15
    X = X[idx,:]
    y = y[idx]
    prediction_surface(models[w], X, y, ax, scatter=False)
fig.tight_layout()
fig.savefig(join(dirplots, 'regression_MLP_zoom'), dpi=500)

## The Influence of Small Land Masses

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,5))
for ax,w in zip(axs, twcols):
    boxplot(
        data=ens,
        x='Area Range',
        y=w,
        hue='Latitude Range',
        palette='flare',
        whis=[0,100],
        ax=ax,
        **box_kws
    )
    ax.set_xticks([])
    ylabel(wrapped(labels[w]), ax)
    ax.set_xlabel(None)
    ax.grid(False)
    ax.get_legend().set_frame_on(False)
    ax.set_xlabel('Land Area\n⟵ small | large ⟶')
fig.tight_layout()
fig.subplots_adjust(top=0.85)
fig.savefig(join(dirplots, 'boxes_total'), dpi=500)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,5))
boxplot(
    data=ens,
    x='Area Range',
    y=np.log10(ens.A / indonesia_area),
    whis=[0,100],
    color='gray',
    **box_kws
)
ax.set_xticks([])
ax.set_xlabel('Land Area\n⟵ small | large ⟶')
ylabel('log$_{10}$ Indonesia\nArea\nFraction', ax)
ax.axhline(0, color='w', linestyle=':', alpha=0.75, zorder=-10)
fig.tight_layout()
fig.subplots_adjust(top=0.85)
fig.savefig(join(dirplots, 'indonesia_area_fracs'), dpi=500)

In [ ]:
npg = ens.groupby(['n', 'p']).sum(numeric_only=True)
ens['whakfrac'], ens['macfrac'] = np.nan, np.nan
for i in ens.index:
    n, p = ens.loc[i,['n','p']]
    for w in ('whak','mac'):
        ens.at[i,w+'frac'] = ens.at[i,w] / npg.at[(n,p), w]
df = ens.drop(list('OTPnpqr') + ['PAR','minlat'] + ['log'+w for w in (mwcols+twcols)], axis=1).copy()
for col in ('Area Range', 'Latitude Range'):
    df[col] = df[col].cat.codes
df.applymap(lambda x: f'{x:.3g}').to_csv(join('..', 'data', 'segmented_weathering_categorized.csv'), index=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,5))
df = ens[(ens.lat < 20) & (ens['Area Range'].cat.codes <= 0)]
for w,ax in zip(twcols, axs):
    scatterplot(
        data=df,
        x='lat',
        y=df[w].apply(np.log),
        ax=ax,
        **scatter_kws
    )
    ylabel('log ' + wrapped(labels[w]), ax)
    ax.set_xlabel(labels['lat'])
fig.tight_layout()
fig.savefig(join(dirplots, 'scatters_weathering_total_tropical'), dpi=500)

In [ ]:
df = ens[(ens.lat < 20) & (ens['Area Range'].cat.codes == 0)].sort_values('lat')
X = pd.concat([df.lat, df.A, np.sqrt(df.A), df.lat*np.sqrt(df.A)], axis=1).values
w = 'whak'
y = df[w].apply(np.log).values
reg = LinearRegression().fit(X,y)
print(reg.coef_)
print(reg.intercept_)
print('r2 =', reg.score(X,y))

fig, axs = plt.subplots(1, 2, figsize=(12,5))

ax = axs[1]
scatterplot(
    x=y,
    y=reg.predict(X).flatten(),
    size=X[:,1],
    hue=X[:,0],
    legend=False,
    palette='flare',
    edgecolor='k',
    ax=ax
)
ax.plot(
    [y.min(), y.max()],
    [y.min(), y.max()],
    linestyle=':',
    color='w',
    linewidth=1,
    zorder=10
)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Actual log ' + unitless(labels[w]))
ylabel('Predicted\nlog ' + unitless(labels[w]), ax)

ax = axs[0]

for i,rA in enumerate(np.linspace(np.sqrt(df.A.min()), np.sqrt(df.A.max()), 5)):
    scatterplot(
        data=df,
        x='lat',
        y=df[w].apply(np.log),
        size='A',
        hue='lat',
        legend=False,
        palette='flare',
        edgecolor='k',
        ax=ax
    )
    X_ = X.copy()
    X_[:,1] = rA**2
    X_[:,2] = rA
    X_[:,3] = X_[:,0]*rA
    ax.plot(df.lat, reg.predict(X_), color='white', alpha=0.5, solid_capstyle='round', linewidth=1+i)
ax.set_xticks([0,20])
ax.set_xlabel(labels['lat'])
ylabel('log ' + wrapped(labels[w]), ax)
fig.savefig(join(dirplots, 'regression_multilinear_mixed'), dpi=500)

In [ ]:
chain = pd.read_feather(join('..', 'data', 'chain.feather'))
chain.columns = ['b', 'beta1', 'beta2', 'beta3', 'beta4', 'g', 'gamma']
N = 8
A = np.linspace(np.sqrt(df.A.min()), np.sqrt(df.A.max()), N)**2
X = np.zeros((N,1))
X = np.hstack([X, (A/A.max()).reshape(-1,1)])
X = np.hstack([X, np.sqrt(X[:,1]).reshape(-1,1)])
X = np.hstack([X, np.zeros((N,1))])
X.shape

In [ ]:
y = (np.dot(X, chain.iloc[:,1:5].values.T) + chain.b.values.reshape(1,-1)).T
y = pd.DataFrame(np.exp(y), columns=range(y.shape[1]))
y = y.melt(var_name='group', value_name='b')
y = y.groupby('group').agg([
    lambda v: np.quantile(v, 0.025),
    lambda v: np.quantile(v, 0.25),
    np.mean,
    lambda v: np.quantile(v, 0.75),
    lambda v: np.quantile(v, 0.975)
])
y

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,4))
ax.fill_between(
    A/indonesia_area,
    y.iloc[:,0],
    y.iloc[:,-1],
    color='C0',
    edgecolor=None,
    alpha=0.25
)
ax.fill_between(
    A/indonesia_area,
    y.iloc[:,1],
    y.iloc[:,-2],
    color='C0',
    edgecolor=None,
    alpha=0.75
)
ax.plot(
    A/indonesia_area,
    y.iloc[:,2],
    color='white',
    linewidth=2,
    solid_capstyle='round'
)
ax.set_xlabel('Area [Indonesia Fraction]')
ylabel(wrapped(labels['whak']), ax)
fig.tight_layout()
fig.savefig(join(dirplots, 'equator_whak_area'), dpi=500)